# SQL Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 5-6 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

Crucial hint: check out the following tutorial:
https://www.dataquest.io/blog/sql-insert-tutorial/


In [2]:
# you must populate this dict with the schools required -> try talking to the teaching team about this


schools = {   
'ironhack' : 10828,
'app-academy' : 10525,
'springboard' : 11035    
}

import re
import pandas as pd
from pandas.io.json import json_normalize
import requests



def get_comments_school(school):
    TAG_RE = re.compile(r'<[^>]+>')
    # defines url to make api call to data -> dynamic with school if you want to scrape competition
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
    #makes get request and converts answer to json
    # url defines the page of all the information, request is made, and information is returned to data variable
    data = requests.get(url).json()
    #converts json to dataframe
    reviews =  pd.DataFrame(data['content']['reviews'])
  
    #aux function to apply regex and remove tags
    def remove_tags(x):
        return TAG_RE.sub('',x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [3]:
# could you write this as a list comprehension? ;)
comments = []

for school in schools.keys():
    print(school)
    comments.append(get_comments_school(school))

comments = pd.concat(comments)

ironhack
app-academy
springboard


In [4]:
comments.head()

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,rawBody,...,queryDate,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school
0,284372,James,False,None,2021.0,False,UX / UI Designer,From teacher to UX/UI designer in 9 weeks,"<span class=""truncatable""><p></p><p>I came int...",<p>I came into the Ironhack full-time remote b...,...,2021-12-21,UX/UI Design Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,I came into the Ironhack full-time remote boot...,ironhack
1,284341,Hugo Michon,False,None,2021.0,True,Cyber security analyst,Exhausting but worth it,"<span class=""truncatable""><p></p><p>I did a fu...",<p>I did a full time remote bootcamp for cyber...,...,2021-12-20,Cyber Security Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,I did a full time remote bootcamp for cybersec...,ironhack
2,284279,Kimera Joseph,False,None,2021.0,False,Data Engineer,Build ETL/ELT pipelines,"<span class=""truncatable""><p></p><p>Pros - Gre...",<p>Pros - Great tutors. Lead teachers and tuto...,...,2021-12-18,Data Analytics Bootcamp,{'image': None},4.7,[],5.0,4.0,5.0,Pros - Great tutors. Lead teachers and tutors ...,ironhack
3,283970,Anonymous,True,None,2021.0,True,Innovation and strategy,Although there are some things I suggest to im...,"<span class=""truncatable""><p></p><p>Pros - Abh...",<p>Pros - Abhi is really talented as a teacher...,...,2021-12-07,Data Analytics Part-Time,{'image': None},3.7,[],4.0,4.0,3.0,Pros - Abhi is really talented as a teacher. H...,ironhack
4,283920,Pekka Tiitinen,False,None,2021.0,False,,Amazing experience,"<span class=""truncatable""><p></p><p>Very in de...",<p>Very in depth content and a hands on approa...,...,2021-12-05,Web Development Part-Time,{'image': None},4.7,[],5.0,5.0,4.0,Very in depth content and a hands on approach ...,ironhack


In [5]:
comments[comments['school']=='ironhack']['program'].unique()

array(['UX/UI Design Bootcamp', 'Cyber Security Bootcamp',
       'Data Analytics Bootcamp', 'Data Analytics Part-Time',
       'Web Development Part-Time', 'Web Development Bootcamp', '', None,
       'UX/UI Design Part-Time', 'Full-time Web Development Bootcamp',
       'Full-time UX/UI Design Bootcamp', 'Part-time Web Development',
       'Part-time UX/UI Design', 'Full-time Web Development Bootcamp ',
       'Part-time UX/UI Design ', 'Web Design'], dtype=object)

In [6]:
comments2=comments.copy()


In [7]:
def program_clean(row):
    if (row['program']=='Full-time UX/UI Design Bootcamp')|(row['program']=='Web Design'):
        return 'UX/UI Design Bootcamp'
    elif (row['program']=='Part-time UX/UI Design')|(row['program']=='Part-time UX/UI Design '):
        return 'UX/UI Design Part-Time'
    elif (row['program']=='Full-time Web Development Bootcamp')|(row['program']=='Full-time Web Development Bootcamp '):
        return 'Web Development Bootcamp'
    elif (row['program']=='Part-time Web Development'):
        return 'Web Development Part-Time'
    elif (row['program']==''):
        return None
    else:
        return row['program']



In [8]:
comments2['program']=comments2.apply(program_clean, axis=1)

In [9]:
comments2[comments2['school']=='ironhack']['program'].unique()

array(['UX/UI Design Bootcamp', 'Cyber Security Bootcamp',
       'Data Analytics Bootcamp', 'Data Analytics Part-Time',
       'Web Development Part-Time', 'Web Development Bootcamp', None,
       'UX/UI Design Part-Time'], dtype=object)

In [10]:
comments2[comments2['school']=='ironhack']['jobTitle'].fillna("No info")

0                              UX / UI Designer
1                        Cyber security analyst
2                                 Data Engineer
3                       Innovation and strategy
4                                              
                         ...                   
1092                                    No info
1093                              iOS Developer
1094                               Entrepreneur
1095                                    No info
1096    Founder, RoR Developer @ Decorissimo.es
Name: jobTitle, Length: 1097, dtype: object

In [16]:
comments2[comments2['school']=='ironhack']['jobTitle'].unique()

array(['UX / UI Designer', 'Cyber security analyst', 'Data Engineer',
       'Innovation and strategy', '', 'Full stack Javascript developer',
       'UX/UI Designer', 'UX UI Designer', 'Full-Stack developer',
       'Web developer', 'Product Designer', 'Full Stack Developer',
       'full stack developer', 'UXUI Designer', 'Frontend Developer',
       'Product designer', 'ux ui designer', 'web dev fullstack',
       'Junior Data Analyst', 'Data Analyst', 'Web Developer',
       'Junior Web Dev', 'UX/UI designer', 'Growth engineer/marketer',
       'Senior Associate', 'Data Analytics', 'Software Engineer',
       'Ux ui designer', 'ux/ui product designer',
       'Fullstack web developer', 'Full stack developer',
       'Development Operations Engineer', 'Web Dev', 'UX Designer',
       'Full-Stack Web Developer', 'Student',
       'Head of Sales / Fullstack developer',
       'Power BI Junior Consultant', 'UX/UI Design',
       'Full Stack Web Developer', 'unemployed', 'frontend devel

In [19]:
print(comments2[comments2['school']=='ironhack']['jobTitle'].str.replace(" ", "No info"))

0                      UXNo info/No infoUINo infoDesigner
1                      CyberNo infosecurityNo infoanalyst
2                                     DataNo infoEngineer
3                     InnovationNo infoandNo infostrategy
4                                                        
                              ...                        
1092                                                 None
1093                                  iOSNo infoDeveloper
1094                                         Entrepreneur
1095                                                 None
1096    Founder,No infoRoRNo infoDeveloperNo info@No i...
Name: jobTitle, Length: 1097, dtype: object


In [17]:
comments2[comments2['school']=='ironhack']['jobTitle'].unique()

array(['UX / UI Designer', 'Cyber security analyst', 'Data Engineer',
       'Innovation and strategy', '', 'Full stack Javascript developer',
       'UX/UI Designer', 'UX UI Designer', 'Full-Stack developer',
       'Web developer', 'Product Designer', 'Full Stack Developer',
       'full stack developer', 'UXUI Designer', 'Frontend Developer',
       'Product designer', 'ux ui designer', 'web dev fullstack',
       'Junior Data Analyst', 'Data Analyst', 'Web Developer',
       'Junior Web Dev', 'UX/UI designer', 'Growth engineer/marketer',
       'Senior Associate', 'Data Analytics', 'Software Engineer',
       'Ux ui designer', 'ux/ui product designer',
       'Fullstack web developer', 'Full stack developer',
       'Development Operations Engineer', 'Web Dev', 'UX Designer',
       'Full-Stack Web Developer', 'Student',
       'Head of Sales / Fullstack developer',
       'Power BI Junior Consultant', 'UX/UI Design',
       'Full Stack Web Developer', 'unemployed', 'frontend devel

In [11]:
comments2[(comments2['school']=='ironhack')&(comments2['program']=='Data Analytics Bootcamp')]['jobTitle'].unique()

array(['Data Engineer', '', 'Junior Data Analyst', 'Data Analyst',
       'Data  analyst',
       'Software Developer Jr., Data Analyst Jr., Structural engineering PhD.',
       'Software Engineer', 'Insights Project Manager',
       'Data Analytics Student', 'Junior data analyst', 'Data analyst',
       'Engenheira de dados',
       'Data Science and Advanced Analytics Manager', None], dtype=object)

In [12]:
comments2[(comments2['school']=='ironhack')&(comments2['program']=='Data Analytics Part-Time')]['jobTitle'].unique()

array(['Innovation and strategy', ''], dtype=object)

In [24]:
comments2[(comments2['school']=='ironhack')&(comments2['program']=='Cyber Security Bootcamp')]['jobTitle'].unique()

array(['Cyber security analyst'], dtype=object)

In [25]:
comments2[(comments2['school']=='ironhack')&(comments2['program']=='Cyber Security Part-Time')]['jobTitle'].unique()

array([], dtype=object)

In [ ]:
def jobTitle_CS_clean(row):
    jobTitle_CS =['analyst']
    for element in jobTitle_CS:
        if (element in row):
            return 1
        else:
            return 0

comments2['Work_inField']=comments2['jobTitle'].apply(jobTitle_CS_clean)

In [26]:
comments2[(comments2['school']=='ironhack')&((comments2['program']=='Web Development Bootcamp')|(comments2['program']=='Web Development Part-Time'))]['jobTitle'].unique()

array(['', 'Full stack Javascript developer', 'Full-Stack developer',
       'Web developer', 'Full Stack Developer', 'full stack developer',
       'Frontend Developer', 'web dev fullstack', 'Web Developer',
       'Junior Web Dev', 'Growth engineer/marketer', 'Senior Associate',
       'Software Engineer', 'Fullstack web developer',
       'Full stack developer', 'Development Operations Engineer',
       'Web Dev', 'Full-Stack Web Developer',
       'Head of Sales / Fullstack developer', 'Full Stack Web Developer',
       'unemployed', 'frontend developer', 'Web Developer junior',
       'web dev', 'web developper', 'CoFounder of Botcave SAS',
       'Fullstack Web Developer', 'Junior Front End Developer',
       'Junior Software Developer', 'React Developer',
       'Full stack web developer', 'Graphic Designer', 'UX-designer',
       'Front-End Developer', 'full stack web developer/teacher',
       'Growth', 'Full-stack Web Developer', 'Full-stack web-developer',
       'Product Ma

In [29]:
comments2[(comments2['school']=='ironhack')&((comments2['jobTitle']=='Founder, Donkies Technologies'))]

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,rawBody,...,queryDate,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school
1035,235469,Ryan Morrison,False,None,2016.0,True,"Founder, Donkies Technologies",Great School,"<span class=""truncatable""><p>Ironhack gave me ...",Ironhack gave me the confidence to build a tec...,...,2017-03-28,Web Development Part-Time,{'image': None},5.0,[],5.0,5.0,5.0,Ironhack gave me the confidence to build a tec...,ironhack


In [27]:
jobTitle_WD =['Senior Associate', 'unemployed', 'Botcave', 'Finance Analyst', 'Program Manager', 'Estudiante', 'Therapist', 'Student', 'student',
'Partner @Orion', 'Founder', 'Student', 'COO at Tender.co', 'Entrepreneur']
def jobTitle_WD_clean(row):
    for element in jobTitle_WD:
        if element in row:
            return 0
        else:
            return 1

comments2['Work_inField']=comments2['jobTitle'].apply(jobTitle_WD_clean)

TypeError: argument of type 'NoneType' is not iterable

In [23]:
def jobTitle_DA_clean(row):
    jobTitle_DA =['Data', 'dados', 'analyst', 'Software', 'Manager', 'Junior', 'Innovation']
    for element in jobTitle_DA:
        if (element in row):
            return 1
        else:
            return 0

comments2['Work_inField']=comments2['jobTitle'].apply(jobTitle_DA_clean)


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
comments2[(comments2['school']=='ironhack')&((comments2['program']=='UX/UI Design Bootcamp')|(comments2['program']=='UX/UI Design Part-Time'))]['jobTitle'].unique()



array(['UX / UI Designer', 'UX/UI Designer', 'UX UI Designer',
       'Product Designer', '', 'UXUI Designer', 'Product designer',
       'ux ui designer', 'UX/UI designer', 'Ux ui designer',
       'ux/ui product designer', 'UX Designer', 'Student', 'UX/UI Design',
       'UX designer', 'Unemployed', 'UI Designer',
       'Learning and Development Specialist', 'Ux/Ui Designer',
       'UX-UI designer', 'Designer', 'UX Designer/UX Writer',
       'UX UI designer', 'ux/ui designer', 'Head of design', None,
       'Junior Product Designer', 'UX DESIGNER', 'UX - UI Designer',
       'UX/UI and graphic designer', 'Teacher Assistant', 'UX UI Design',
       'Freelancing UX/UI Designer &amp; Travel agent', 'UX/UI',
       'UX &amp; UI Designer', 'UX DEsigner', 'co-founder at Leemur',
       'Ux/UI Designer', 'Creative Designer',
       'UX/UI Design Teacher Assistant', 'UX/UI Designer ',
       'Creative Team Lead', 'Creative Manager',
       'Creative Development Manager', 'Interaction Desi

In [ ]:
jobTitle_UXUI =['Student', 'student', 'Unemployed', 'Leemur', 'Owner']
def jobTitle_UXUI_clean(row):
    for element in jobTitle_UXUI:
        if element in row:
            return 0
        else:
            return 1

comments2['Work_inField']=comments2['jobTitle'].apply(jobTitle_UXUI_clean)

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
comments2[(comments2['school']=='ironhack')&((comments2['program']=='UX/UI Design Bootcamp')|(comments2['program']=='UX/UI Design Part-Time'))]['jobTitle'].unique()


In [ ]:
comments2[comments2['school']=='ironhack']['program'].unique()

array(['UX/UI Design Bootcamp', 'Cyber Security Bootcamp',
       'Data Analytics Bootcamp', 'Data Analytics Part-Time',
       'Web Development Part-Time', 'Web Development Bootcamp', None,
       'UX/UI Design Part-Time'], dtype=object)

In [ ]:
comments2[comments2['school']=='ironhack']['graduatingYear'].unique()

array([2021., 2020., 2018., 2019., 2017., 2011., 2016., 2015., 2014.])

In [ ]:
comments2[comments2['school']=='ironhack']['graduatingYear'].dtype

dtype('float64')

In [ ]:
def price_program(row):
    if row['graduatingYear'] == 2021.0:
        if row['program'] == 'UX/UI Design Bootcamp':
            return 12.000
        elif row['program'] == 'Cyber Security Bootcamp':
            return 7.500
        elif row['program'] == 'Data Analytics Bootcamp':
            return 7.500
        elif row['program'] == 'Web Development Bootcamp':
            return 12.000
        elif row['program'] == 'UX/UI Design Part-Time':
            return 13.000
        elif row['program'] == 'Cyber Security Part-Time':
            return 7.500
        elif row['program'] == 'Data Analytics Part-Time':
            return 8.000
        elif row['program'] == 'Web Development Part-Time':
            return 13.000
    else:
        return 'check price'

In [ ]:
comments2['price']=comments2.apply(price_program, axis=1)

In [ ]:
comments2['price'].unique()

array([12.0, 7.5, 8.0, 13.0, None, 'check price'], dtype=object)

In [ ]:
comments2.head(2)

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,rawBody,...,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school,price
0,284372,James,False,None,2021.0,False,UX / UI Designer,From teacher to UX/UI designer in 9 weeks,"<span class=""truncatable""><p></p><p>I came int...",<p>I came into the Ironhack full-time remote b...,...,UX/UI Design Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,I came into the Ironhack full-time remote boot...,ironhack,12.0
1,284341,Hugo Michon,False,None,2021.0,True,Cyber security analyst,Exhausting but worth it,"<span class=""truncatable""><p></p><p>I did a fu...",<p>I did a full time remote bootcamp for cyber...,...,Cyber Security Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,I did a full time remote bootcamp for cybersec...,ironhack,7.5


In [ ]:
comments2[(comments2['school']=='ironhack')|comments2['program']=='ironhack']['jobTitle'].unique()

array([], dtype=object)

In [ ]:
from pandas.io.json import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])
    
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school
    

    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools.items():
    print(school)
    a,b,c,d = get_school_info(school,id)
    
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)



ironhack


<ipython-input-160-7111c633f1e2>:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  locations_df = json_normalize(locations)


app-academy
springboard


In [ ]:
locations_list

[      id               description  country.id   country.name country.abbrev  \
 0  15901           Berlin, Germany        57.0        Germany             DE   
 1  16022       Mexico City, Mexico        29.0         Mexico             MX   
 2  16086    Amsterdam, Netherlands        59.0    Netherlands             NL   
 3  16088         Sao Paulo, Brazil        42.0         Brazil             BR   
 4  16109             Paris, France        38.0         France             FR   
 5  16375  Miami, FL, United States         1.0  United States             US   
 6  16376             Madrid, Spain        12.0          Spain             ES   
 7  16377          Barcelona, Spain        12.0          Spain             ES   
 8  16709          Lisbon, Portugal        28.0       Portugal             PT   
 9  17233                    Online         NaN            NaN            NaN   
 
    city.id    city.name city.keyword  state.id state.name state.abbrev  \
 0  31156.0       Berlin       b

In [ ]:
locations = pd.concat(locations_list)
locations

,id,description,country.id,country.name,country.abbrev,city.id,city.name,city.keyword,state.id,state.name,state.abbrev,state.keyword,school,school_id
0,15901,"Berlin, Germany",57.0,Germany,DE,31156.0,Berlin,berlin,NaN,NaN,NaN,NaN,ironhack,10828
1,16022,"Mexico City, Mexico",29.0,Mexico,MX,31175.0,Mexico City,mexico-city,NaN,NaN,NaN,NaN,ironhack,10828
2,16086,"Amsterdam, Netherlands",59.0,Netherlands,NL,31168.0,Amsterdam,amsterdam,NaN,NaN,NaN,NaN,ironhack,10828
3,16088,"Sao Paulo, Brazil",42.0,Brazil,BR,31121.0,Sao Paulo,sao-paulo,NaN,NaN,NaN,NaN,ironhack,10828
4,16109,"Paris, France",38.0,France,FR,31136.0,Paris,paris,NaN,NaN,NaN,NaN,ironhack,10828
5,16375,"Miami, FL, United States",1.0,United States,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
6,16376,"Madrid, Spain",12.0,Spain,ES,31052.0,Madrid,madrid,NaN,NaN,NaN,NaN,ironhack,10828
7,16377,"Barcelona, Spain",12.0,Spain,ES,31170.0,Barcelona,barcelona,NaN,NaN,NaN,NaN,ironhack,10828
8,16709,"Lisbon, Portugal",28.0,Portugal,PT,31075.0,Lisbon,lisbon,NaN,NaN,NaN,NaN,ironhack,10828
9,17233,Online,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Online,Online,online,ironhack,10828


In [ ]:
courses = pd.concat(courses_list)
courses

,courses,school,school_id
0,Cyber Security Bootcamp,ironhack,10828
1,Cybersecurity Part-Time,ironhack,10828
2,Data Analytics Bootcamp,ironhack,10828
3,Data Analytics Part-Time,ironhack,10828
4,UX/UI Design Bootcamp,ironhack,10828
5,UX/UI Design Part-Time,ironhack,10828
6,Web Development Bootcamp,ironhack,10828
7,Web Development Part-Time,ironhack,10828
0,Bootcamp Prep,app-academy,10525
1,Software Engineer Track: In-Person,app-academy,10525


In [ ]:
badges = pd.concat(badges_list)
badges

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
0,Available Online,available_online,<p>School offers fully online courses</p>,app-academy,10525
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,app-academy,10525
0,Available Online,available_online,<p>School offers fully online courses</p>,springboard,11035
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,springboard,11035
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035


In [ ]:
# any data cleaning still missing here? take a look at the description
schools = pd.concat(schools_list)
schools

,website,description,LogoUrl,school,school_id
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,appacademy.io,"<span class=""truncatable""><p>App Academy is a ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy,10525
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035
